In [ ]:
#1.importing libraries

import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import nltk
nltk.download('stopwords')

#printing the stopwords in english language
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#2.preprocessing

#loading the data from csv file to pandas data frame
movie_data = pd.read_csv("/content/IMDB Dataset.csv")

#printing the no of rows and columns
movie_data.shape

#printing the first five rows
movie_data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
# #counting the no of missing values
movie_data.isnull().sum()

,0
review,0
sentiment,0


In [ ]:
#checking the distribution of sentiment column
movie_data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


**stemming**

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def preprocess_and_tokenize(content):
    # Removing non-alphabet characters and converting to lowercase
    content = re.sub('[^a-zA-Z]', ' ', content).lower()
    # Splitting and removing stopwords
    tokens = [port_stem.stem(word) for word in content.split() if word not in stopwords.words('english')]
    return tokens

In [ ]:
# Tokenize the review
movie_data['tokens'] = movie_data['review'].apply(preprocess_and_tokenize)

In [ ]:
# Separating the data and labels
x = movie_data['tokens'].values
y = movie_data['sentiment'].values

In [ ]:
movie_data.head()

,review,sentiment,tokens
0,One of the other reviewers has mentioned that ...,positive,"[one, review, mention, watch, oz, episod, hook..."
1,A wonderful little production. <br /><br />The...,positive,"[wonder, littl, product, br, br, film, techniq..."
2,I thought this was a wonderful way to spend ti...,positive,"[thought, wonder, way, spend, time, hot, summe..."
3,Basically there's a family where a little boy ...,negative,"[basic, famili, littl, boy, jake, think, zombi..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[petter, mattei, love, time, money, visual, st..."


In [ ]:
print(x)

[list(['one', 'review', 'mention', 'watch', 'oz', 'episod', 'hook', 'right', 'exactli', 'happen', 'br', 'br', 'first', 'thing', 'struck', 'oz', 'brutal', 'unflinch', 'scene', 'violenc', 'set', 'right', 'word', 'go', 'trust', 'show', 'faint', 'heart', 'timid', 'show', 'pull', 'punch', 'regard', 'drug', 'sex', 'violenc', 'hardcor', 'classic', 'use', 'word', 'br', 'br', 'call', 'oz', 'nicknam', 'given', 'oswald', 'maximum', 'secur', 'state', 'penitentari', 'focus', 'mainli', 'emerald', 'citi', 'experiment', 'section', 'prison', 'cell', 'glass', 'front', 'face', 'inward', 'privaci', 'high', 'agenda', 'em', 'citi', 'home', 'mani', 'aryan', 'muslim', 'gangsta', 'latino', 'christian', 'italian', 'irish', 'scuffl', 'death', 'stare', 'dodgi', 'deal', 'shadi', 'agreement', 'never', 'far', 'away', 'br', 'br', 'would', 'say', 'main', 'appeal', 'show', 'due', 'fact', 'goe', 'show', 'dare', 'forget', 'pretti', 'pictur', 'paint', 'mainstream', 'audienc', 'forget', 'charm', 'forget', 'romanc', 'oz', '

In [ ]:
print(y)

['positive' 'positive' 'positive' ... 'negative' 'negative' 'negative']


**Splitting the data into training data and test data**

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)


In [ ]:
print(x.shape,x_train.shape,x_test.shape)

(50000,) (40000,) (10000,)


**vectorization**

In [ ]:
import gensim
from gensim.models import Word2Vec

In [ ]:
#5. Word2Vec Embeddings
# Training Word2Vec model on the training data
w2v_model = Word2Vec(sentences=x_train.tolist(), vector_size=100, window=5, min_count=1, sg=1)

# Function to get the average word vector for each review
def get_average_word2vec(tokens, model, vector_size):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(vector_size)

# Creating average word vectors for each review in training and test sets
x_train_vec = np.array([get_average_word2vec(tokens, w2v_model, 100) for tokens in x_train])
x_test_vec = np.array([get_average_word2vec(tokens, w2v_model, 100) for tokens in x_test])

**Training the Machine learning model**

Logistic Regression

In [ ]:
lr = LogisticRegression()
lr.fit(x_train_vec,y_train)

LogisticRegression()

In [ ]:
x_train_prediction = lr.predict(x_train_vec)
training_data_accuracy = accuracy_score(x_train_prediction,y_train)

In [ ]:
print('Accuracy score of the training data: ',training_data_accuracy*100)

Accuracy score of the training data:  87.07249999999999


Naive Bayes

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Assuming x_train_vec is your feature matrix with potential negative values
scaler = MinMaxScaler()
x_train_vec_scaled = scaler.fit_transform(x_train_vec)
x_test_vec_scaled = scaler.transform(x_test_vec)

# Now fit the MultinomialNB model
classifier = MultinomialNB()
classifier.fit(x_train_vec_scaled,y_train)

MultinomialNB()

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
x_train_prediction_nb = classifier.predict(x_train_vec_scaled)
training_datas_accuracy_nb = accuracy_score(x_train_prediction_nb,y_train)

In [ ]:
print('Accuracy score of the training data: ',training_datas_accuracy_nb*100)

Accuracy score of the training data:  76.7475
